In [3]:
import numpy as np
import keras
import torch
from kymatio.torch import Scattering1D
import sys
sys.path.append("../src")
import cnn
import ftm_ver2 as ftm2
import hcqt
import soundfile as sf

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv1D, AveragePooling1D, Conv2D, MaxPooling2D,ReLU,AveragePooling2D
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model #save and load models
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
import IPython.display as ipd
import librosa

## reshape different features
1. CQT
2. VQT
3. HCQT
4. Time scattering 1st order
5. Time scattering 2nd order

set consistent parameters for different features

In [1]:
j = 14
q = 1
N = 2**16
sr = 22050
n_oct = j
t = 2**11
b = q
fmin = 0.4*sr*2**(-j)
comp_hcqt = hcqt.HCQT(sr,bins_per_octave=b,harmonics=[0.5,1,2,3,4,5],
                 n_octaves=n_oct-2,f_min=fmin,hop_length=256)

NameError: name 'hcqt' is not defined

In [5]:
timesc = Scattering1D(
        J = j, #scale, how big the biggest time support (for the lowest freqeuncy), where center frequencies are?
        shape = (N, ),
        Q = q,
        T=t, 
        max_order=1
        )
timesc2 = Scattering1D(
        J = j, #scale, how big the biggest time support (for the lowest freqeuncy), where center frequencies are?
        shape = (N, ),
        Q = q,
        T=t, 
        max_order=2
        )

compute features

In [ ]:
y1,sr = sf.read("/home/han/data/drum_data/val/22222_sound.wav")
y1 = torch.Tensor(y1)

In [34]:
cqt = librosa.cqt(y1.numpy(),sr=sr,n_bins=n_oct*b,fmin=fmin,hop_length=256,bins_per_octave=b) 
v_cqt = np.abs(librosa.vqt(y1.numpy(),sr=sr,n_bins=n_oct*b,fmin=fmin,gamma=None,bins_per_octave=b,hop_length=256))
h_cqt = comp_hcqt.compute_hcqt(y1.numpy(),sr)
sc = timesc(y1) #after 2**j averaging window
sc2 = timesc2(y1)

In [35]:
cqt.shape,v_cqt.shape,h_cqt.shape,sc.shape,sc2.shape

((14, 257),
 (14, 256),
 (6, 12, 257),
 torch.Size([16, 32]),
 torch.Size([118, 32]))

In [7]:
import os
import pandas as pd
data_dir = "/home/han/data/drum_data/"
csv_path = os.path.join(data_dir,"annotations","train_param_v2.csv")
df = pd.read_csv(csv_path)
sample_ids = df.values[:, 0]

## wave2shape model
1. fully connected CNN model
2. #layers ~ log(# scattering coefficients $\lambda$)
3. input: scattering transform of each percussive sounds (number of audio N, scattering coeff $\lambda$, time K)
4. output: 5 physical parameters $\theta = {\tau, p, D, \alpha, w}$

what invariance do i want?
1. equivariance to pitch shift, so no pooling along frequency axis
2. invariant to time shift, can be pooled along time axis. maximum invariance, half of the audio length..
3. equivariance to time decay.


In [6]:
y = ftm2.getsounds_imp_linear_nonorm(10,10,0.4,0.6,0.03,0.3,100*2*np.pi,
                                 0.02,0.03,np.pi,1,44100)
S = timesc(torch.Tensor(y))
nrow, ncol = S.shape 
naudio = 1         # number of audio in batch
nchan_in = 1       # number of input channels. 
input_shape = (nrow,ncol,nchan_in)  # input shape of 1 image
batch_shape = (naudio,nrow,ncol,nchan_in)  # shape of image batch

S = S.reshape(batch_shape)
print(S.shape)
#this way convolution is done along time axis, each kernel looks at all frequencies at once
S_input_shape = (S.shape[1],S.shape[2],nchan_in) #f,t,#channels

S_input = keras.layers.Input(shape=S_input_shape)
kernel_size = (8,)
nchan_out = 16

x = BatchNormalization()(S_input)
x = Conv1D(filters=128,
        kernel_size=(5,), padding="same",name='conv1')(x) #convolve along the last dimension before #channel
x = AveragePooling2D(pool_size=(1,4),padding="valid")(x)
x = Conv1D(filters=64,
          kernel_size=(3,),padding="same",name="conv1_2")(x)
x = AveragePooling2D(pool_size=(1,4),padding="valid")(x)
x = BatchNormalization()(x)
x = Conv1D(filters=64,
          kernel_size=(3,),padding="same",name="conv1_3")(x)
x = BatchNormalization()(x)
x = Conv1D(filters=8,
          kernel_size=(1,),padding="same",name="conv1_4")(x)
x = BatchNormalization()(x)

x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
#what activation should be chosen for last layer, for regression problem? should be a linear function
x = Dense(5, activation="linear")(x)
#x = AveragePooling2D(pool_size=4,padding="valid")(x)


model = Model(inputs=[S_input], outputs=x)


model.summary()

torch.Size([1, 16, 32, 1])


2022-05-02 23:40:12.707865: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 23:40:13.022088: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 23:40:13.023884: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 23:40:13.042269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 23:40:13.043759: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16, 32, 1)]       0         
                                                                 
 batch_normalization (BatchN  (None, 16, 32, 1)        4         
 ormalization)                                                   
                                                                 
 conv1 (Conv1D)              (None, 16, 32, 128)       768       
                                                                 
 average_pooling2d (AverageP  (None, 16, 8, 128)       0         
 ooling2D)                                                       
                                                                 
 conv1_2 (Conv1D)            (None, 16, 8, 64)         24640     
                                                                 
 average_pooling2d_1 (Averag  (None, 16, 2, 64)        0     

In [11]:
y = ftm2.getsounds_imp_linear_nonorm(10,10,0.4,0.6,0.03,0.3,100*2*np.pi,
                                 0.02,0.03,np.pi,1,44100)
S = timesc(torch.Tensor(y))
nrow, ncol = S.shape 
print(S.shape)
naudio = 1         # number of audio in batch
nchan_in = 1       # number of input channels. 
input_shape = (nrow,ncol,nchan_in)  # input shape of 1 image
batch_shape = (naudio,nrow,ncol,nchan_in)  # shape of image batch

S = S.reshape(batch_shape)
print(S.shape)
#this way convolution is done along time axis, each kernel looks at all frequencies at once
S_input_shape = (S.shape[2],S.shape[1]) #f,t,#channels

S_input = keras.layers.Input(shape=S_input_shape)
kernel_size = (8,)
nchan_out = 16

x = BatchNormalization()(S_input)
x = Conv1D(filters=128,
        kernel_size=(5,), padding="same",name='conv1')(x) #convolve along the last dimension before #channel
x = AveragePooling1D(pool_size=(4,),padding="valid")(x)
x = Conv1D(filters=64,
          kernel_size=(3,),padding="same",name="conv1_2")(x)
x = AveragePooling1D(pool_size=(4,),padding="valid")(x)
x = BatchNormalization()(x)
x = Conv1D(filters=64,
          kernel_size=(3,),padding="same",name="conv1_3")(x)
x = BatchNormalization()(x)
x = Conv1D(filters=8,
          kernel_size=(1,),padding="same",name="conv1_4")(x)
x = BatchNormalization()(x)

x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
#what activation should be chosen for last layer, for regression problem? should be a linear function
x = Dense(5, activation="linear")(x)
#x = AveragePooling2D(pool_size=4,padding="valid")(x)


model = Model(inputs=[S_input], outputs=x)


model.summary()

torch.Size([16, 32])
torch.Size([1, 16, 32, 1])
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 16)]          0         
                                                                 
 batch_normalization_17 (Bat  (None, 32, 16)           64        
 chNormalization)                                                
                                                                 
 conv1 (Conv1D)              (None, 32, 128)           10368     
                                                                 
 average_pooling1d_4 (Averag  (None, 8, 128)           0         
 ePooling1D)                                                     
                                                                 
 conv1_2 (Conv1D)            (None, 8, 64)             24640     
                                                                 
 average_po

In [24]:
h_cqt = comp_hcqt.compute_hcqt(y1.numpy(),sr)y = ftm2.getsounds_imp_linear_nonorm(10,10,0.4,0.6,0.03,0.3,100*2*np.pi,
                                 0.02,0.03,np.pi,1,44100)
S = timesc(torch.Tensor(y))
nrow, ncol = S.shape 
naudio = 1         # number of audio in batch
nchan_in = 1       # number of input channels. 
input_shape = (nrow,ncol,nchan_in)  # input shape of 1 image
batch_shape = (naudio,nrow,ncol,nchan_in)  # shape of image batch

S = S.reshape(batch_shape)
print(S.shape)
#this way convolution is done along time axis, each kernel looks at all frequencies at once
S_input_shape = (S.shape[1],S.shape[2],nchan_in) #f,t,#channels

S_input = keras.layers.Input(shape=S_input_shape)
kernel_size = (8,)
nchan_out = 16

x = BatchNormalization()(S_input)
x = Conv1D(filters=128,
        kernel_size=(5,), padding="same",name='conv1')(x) #convolve along the last dimension before #channel
x = AveragePooling2D(pool_size=(1,4),padding="valid")(x)
x = Conv1D(filters=64,
          kernel_size=(3,),padding="same",name="conv1_2")(x)
x = AveragePooling2D(pool_size=(1,4),padding="valid")(x)
x = BatchNormalization()(x)
x = Conv1D(filters=64,
          kernel_size=(3,),padding="same",name="conv1_3")(x)
x = BatchNormalization()(x)
x = Conv1D(filters=8,
          kernel_size=(1,),padding="same",name="conv1_4")(x)
x = BatchNormalization()(x)

x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
#what activation should be chosen for last layer, for regression problem? should be a linear function
x = Dense(5, activation="linear")(x)
#x = AveragePooling2D(pool_size=4,padding="valid")(x)


model = Model(inputs=[S_input], outputs=x)


model.summary()
print(h_cqt.shape)
nchan_in,nrow, ncol = h_cqt.shape 
naudio = 1         # number of images in batch
input_shape = (nrow,ncol,nchan_in)  # input shape of 1 image
batch_shape = (naudio,nrow,ncol,nchan_in)  # shape of image batch
h_cqt = h_cqt.reshape(batch_shape)
#2D convolution is done along time and frequency axis, each kernel looks at 6 channels at once
hcqt_input_shape = (h_cqt.shape[1],h_cqt.shape[2],h_cqt.shape[3]) 

hcqt_input = keras.layers.Input(shape=hcqt_input_shape)
nchan_out = 16
x = BatchNormalization()(hcqt_input)
x = Conv2D(filters=128,
        kernel_size=(5,5), padding="same",name='conv1')(x) #convolve along the last dimension before #channel
x = AveragePooling2D(pool_size=(1,16),padding="valid")(x)
x = Conv2D(filters=64,
        kernel_size=(3,3), padding="same",name='conv2')(x)
x = AveragePooling2D(pool_size=(1,4),padding="valid")(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64,
        kernel_size=(3,3), padding="same",name='conv3')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=8,
        kernel_size=(1,1), padding="same",name='conv4')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(5, activation="linear")(x)
model = Model(inputs=[hcqt_input], outputs=x)

model.summary()

(6, 160, 257)
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 160, 257, 6)]     0         
                                                                 
 batch_normalization_15 (Bat  (None, 160, 257, 6)      24        
 chNormalization)                                                
                                                                 
 conv1 (Conv2D)              (None, 160, 257, 128)     19328     
                                                                 
 average_pooling2d_5 (Averag  (None, 160, 16, 128)     0         
 ePooling2D)                                                     
                                                                 
 conv2 (Conv2D)              (None, 160, 16, 64)       73792     
                                                                 
 average_pooling2d_6 (Averag  (None, 160, 4, 

In [30]:
Scattering1D?

In [10]:
cqt = librosa.cqt(y1.numpy(),sr=sr,n_bins=(n_oct)*b,fmin=fmin,hop_length=256,bins_per_octave=b) 
print(cqt.shape)
nrow, ncol = cqt.shape 
naudio = 1         # number of images in batch
input_shape = (nrow,ncol,nchan_in)  # input shape of 1 image
nchan_in = 1
batch_shape = (naudio,nrow,ncol,nchan_in)  # shape of image batch
cqt = cqt.reshape(batch_shape)
#2D convolution is done along time and frequency axis, each kernel looks at 6 channels at once
cqt_input_shape = (cqt.shape[2],cqt.shape[1],nchan_in)
print(cqt_input_shape,cqt.shape)

cqt_input = keras.layers.Input(shape=cqt_input_shape)
kernel_size = (8,8) #frequency and time
nchan_out = 16
x = BatchNormalization()(cqt_input)
x = Conv2D(filters=nchan_out,
          kernel_size=kernel_size, padding="same",name="conv2d")(x)
#pool time resolution to match scattering
x = AveragePooling2D(pool_size=(16,1),strides=None, padding="valid")(x) #average across 4 

x = Conv2D(filters=1,kernel_size=(5,1),padding="same",name="squish")(x) 
model = Model(inputs=[cqt_input], outputs=x)

model.summary()

(192, 257)
(257, 192, 1) (1, 192, 257, 1)
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 257, 192, 1)]     0         
                                                                 
 batch_normalization_2 (Batc  (None, 257, 192, 1)      4         
 hNormalization)                                                 
                                                                 
 conv2d (Conv2D)             (None, 257, 192, 16)      1040      
                                                                 
 average_pooling2d_1 (Averag  (None, 16, 192, 16)      0         
 ePooling2D)                                                     
                                                                 
 squish (Conv2D)             (None, 16, 192, 1)        81        
                                                                 
Total params: 1,1

## Build two experiments and move them into training scripts. start running them!

1. load scattering features
2. make model
3. randomly choose samples and train fit

In [36]:
import tensorflow as tf
#verify if using GPU
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-04-16 10:13:08.750187: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


True

2022-04-16 10:13:08.888019: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 10:13:08.888728: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 10:13:10.025512: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 10:13:10.026260: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 10:13:10.026957: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [38]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [39]:
df_train = pd.read_csv("/home/han/data/drum_data/annotations/train_param_v2.csv")
df_test = pd.read_csv("/home/han/data/drum_data/annotations/test_param_v2.csv")

In [42]:
params = df_train.values[:,1:-1]

In [13]:
# normalization of the physical parameters
params = df.values[:,1:-1]
scaler = MinMaxScaler()
scaler.fit(params)
params_normalized = scaler.transform(params)

In [2]:
import pickle
import math
import os 

pkl_dir = '/home/han/data/drum_data/han2022features-pkl/'
J = 12
Q = 1
feature = "scattering_o1"
fold_str = "train"
pickle_name = "_".join(
        [feature,
         "fold-"+str(fold_str),"J-" + str(J).zfill(2), "Q-" + str(Q).zfill(2)]
    )

pkl_path_train = os.path.join(pkl_dir,feature,pickle_name+".pkl")
pkl_train = open(pkl_path_train,'rb')
Sy_train,y_train = pickle.load(pkl_train)


In [3]:
J = 12
Q = 11
feature = "scattering_o1"
fold_str = "val"
pickle_name = "_".join(
        [feature,
         "fold-"+str(fold_str),"J-" + str(J).zfill(2), "Q-" + str(Q).zfill(2)]
    )

pkl_path_val = os.path.join(pkl_dir,feature,pickle_name+".pkl")
pkl_val = open(pkl_path_val,'rb')
Sy_val,y_val = pickle.load(pkl_val)


In [4]:
Sy_train.shape,y_train.shape,Sy_val.shape,y_val.shape

((82224, 32, 14), (82224, 5), (7776, 32, 117), (7776, 5))

In [5]:
import cnn
steps_per_epoch = 50
bs = 64
m = bs*steps_per_epoch
j = 12
q = 1
model = cnn.create_model_conv1d(j,q,Sy_train[:m,:,:])
model.summary()

2022-04-16 11:23:44.114576: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 11:23:44.123989: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 11:23:44.124809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 11:23:44.126606: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 11:23:44.127317: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14, 32, 1)]       0         
                                                                 
 batch_normalization (BatchN  (None, 14, 32, 1)        4         
 ormalization)                                                   
                                                                 
 conv1 (Conv1D)              (None, 14, 32, 128)       1152      
                                                                 
 average_pooling2d (AverageP  (None, 14, 8, 128)       0         
 ooling2D)                                                       
                                                                 
 batch_normalization_1 (Batc  (None, 14, 8, 128)       512       
 hNormalization)                                                 
                                                             

log scale training features, normalize ground truth 

In [70]:
#log scale omega and p and D 
y_train = y_train.astype('float')
for idx in [0,2,3]:
	y_train[:,idx] = [np.log10(i) for i in y_train[:,idx]]

# normalization of the physical parameters
scaler = MinMaxScaler()
scaler.fit(y_train)
y_train_normalized = scaler.transform(y_train)

#log scale the input
eps = 1e-11
Sy_train_log2 = np.log1p(((Sy_train>0)*Sy_train)/eps)


In [71]:
Sy_train_log2.shape

naudio,nrow, ncol = Sy_train_log2.shape 
nchan_in = 1       # number of input channels. 
input_shape = (nrow,ncol,nchan_in)  # input shape of 1 image
batch_shape = (naudio,nrow,ncol,nchan_in)  # shape of image batch

Sy_train_log2 = Sy_train_log2.reshape(batch_shape)
print(Sy_train_log2.shape)
#this way convolution is done along time axis, each kernel looks at all frequencies at once
S_input_shape = (Sy_train_log2.shape[1],Sy_train_log2.shape[2],nchan_in) #f,t,#channels

S_input = keras.layers.Input(shape=S_input_shape)
kernel_size = (8,)
nchan_out = 16

(82224, 32, 14)

In [79]:

best_validation_loss = np.inf
zoom_factor = 1 #only look at part of the time data
n = Sy_train.shape[0]
shape_time = round(Sy_train.shape[1] * zoom_factor)
steps_per_epoch = 50
bs = 64
m = bs*steps_per_epoch
idx = np.arange(0,n,1)
val_loss=[]
train_loss = []
test_loss = []
#save_log = os.path.join(trial_dir,pickle_name+"_score.pkl")
#model_adjustable.summary()
print('Start fitting the model...')
for epoch in range(30):
	np.random.shuffle(idx)
	Sy_temp = Sy_train_log2[idx[:m],:shape_time,:]
	y_temp = y_train_normalized[idx[:m],:]

	hist = model.fit(Sy_temp,
				y_temp,
				epochs=1,
				verbose=2,
				batch_size=bs,
				validation_data = (Sy_train_log2[:,-shape_time:,:],y_train_normalized),
				use_multiprocessing=False)

	validation_loss = hist.history['val_loss'][0]

	test_loss.append(model.evaluate(Sy_train_log2,y_train_normalized)[0])
	val_loss.append(validation_loss)
	train_loss.append(hist.history['loss'][0])


Start fitting the model...


2022-04-15 17:09:37.361343: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2022-04-15 17:09:39.769103: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-15 17:09:39.769485: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-15 17:09:39.769514: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-04-15 17:09:39.769883: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-15 17:09:39.769960: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-04-15 17:09:41.577303: W tensorflow/core/framework/cpu_all

50/50 - 10s - loss: 0.2883 - mse: 0.2883 - val_loss: 0.6052 - val_mse: 0.6052 - 10s/epoch - 209ms/step
   1/2570 [..............................] - ETA: 2:25 - loss: 0.6361 - mse: 0.6361

2022-04-15 17:09:46.014422: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 147345408 exceeds 10% of free system memory.
2022-04-15 17:09:46.148620: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 147345408 exceeds 10% of free system memory.


2570/2570 [==============================] - 8s 3ms/step - loss: 0.6052 - mse: 0.6052


2022-04-15 17:09:54.704591: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 147345408 exceeds 10% of free system memory.


50/50 - 4s - loss: 0.0823 - mse: 0.0823 - val_loss: 0.3800 - val_mse: 0.3800 - 4s/epoch - 85ms/step
2570/2570 [==============================] - 8s 3ms/step - loss: 0.3800 - mse: 0.3800
50/50 - 4s - loss: 0.0517 - mse: 0.0517 - val_loss: 0.6434 - val_mse: 0.6434 - 4s/epoch - 86ms/step
 872/2570 [=========>....................] - ETA: 5s - loss: 0.6442 - mse: 0.6442

KeyboardInterrupt: 

In [16]:
def feature_sampler(df,params_normalized,idx,path_to_folder,J,Q,order):
    """
    output a {input, ground truth} pair for the designated audio sample
    """
    i=idx
    y=params_normalized[i,:] #df.values[i,1:-1]
    path_to_audio = os.path.join(path_to_folder,str(df.values[i,0])+"_sound.wav") 
    x,fs=librosa.load(path_to_audio)
    Sy = getsc_new(x,J,Q,order)
    while True:
        #np.random.shuffle(idx)
        #i=idx[0]
        yield {'input': Sy,'y': y}

        
def data_generator(df, params_normalized, path_to_folder, J, Q, order, batch_size, idx, active_streamers,
                        rate, random_state=12345678):
    """
    use streamers to output a batch of {input groundtruth} pairs. 
    """
    seeds = []
    for i in idx:
        streamer = pescador.Streamer(feature_sampler, df, params_normalized, i,path_to_folder,J,Q,order)
        seeds.append(streamer)

    # Randomly shuffle the seeds
    random.shuffle(seeds)

    mux = pescador.StochasticMux(seeds, active_streamers, rate=rate, random_state=random_state)

    return pescador.maps.buffer_stream(mux, batch_size)

In [ ]:
## first run with small number of training
epochs=12
batch_size=32
random_state=12345678
active_streamers=1024
path_to_train = "/scratch/hh2263/drum_dataset/train/"
path_to_test = "/scratch/hh2263/drum_dataset/test/"
J = 8
Q = 1
order = 1 # remember to go to order 2 eventually
train_idx = df_train.values[0,:100]
test_idx = df_test.values[0,:30]
train_batches=data_generator(df,params_normalized, path_to_train,J, Q, order, batch_size, train_idx,active_streamers,rate=64,random_state=random_state)
test_batches=data_generator(df,params_normalized, path_to_train,J, Q, order, batch_size, train_idx,active_streamers,rate=64,random_state=random_state)
steps_per_epoch = len(train_idx) // batch_size

In [ ]:
fname = random.choice(os.listdir(path_to_train))
rand_audio = os.path.join(path_to_train,fname)
y = librosa.load(rand_audio)
Sy = getsc_new(y,J,Q,order).T
nrow, ncol = Sy.shape 
naudio = batch_size         # number of images in batch
nchan_in = 1       # number of input channels.  1 since it is BW
input_shape = (nrow,ncol,nchan_in)  # input shape of 1 image
batch_shape = (naudio,nrow,ncol,nchan_in)  # shape of image batch
x = S.reshape(batch_shape)

In [ ]:
#preliminary test
hist = model.fit_generator(
        pescador.maps.keras_tuples(train_batches, 'input', 'y'),
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=pescador.maps.keras_tuples(test_batches, 'input', 'y'),
        validation_steps=1024,
        verbose=1
        )